In [2]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd 

from pydataset import data

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

import graphviz
from graphviz import Graph

import acquire

from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [72]:
telco = acquire.get_telco_data()
df = telco
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,...,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,...,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,...,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [73]:
df.isnull().mean()
#No nulls

payment_type_id             0.0
internet_service_type_id    0.0
contract_type_id            0.0
customer_id                 0.0
gender                      0.0
senior_citizen              0.0
partner                     0.0
dependents                  0.0
tenure                      0.0
phone_service               0.0
multiple_lines              0.0
online_security             0.0
online_backup               0.0
device_protection           0.0
tech_support                0.0
streaming_tv                0.0
streaming_movies            0.0
paperless_billing           0.0
monthly_charges             0.0
total_charges               0.0
churn                       0.0
contract_type               0.0
internet_service_type       0.0
payment_type                0.0
dtype: float64

In [74]:
df.shape

(7043, 24)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

In [76]:
df.churn.value_counts()

#### Initial Focus

No     5174
Yes    1869
Name: churn, dtype: int64

In [77]:
df.contract_type.value_counts()

Month-to-month    3875
Two year          1695
One year          1473
Name: contract_type, dtype: int64

In [84]:
df.internet_service_type.value_counts()

Fiber optic    3096
DSL            2421
None           1526
Name: internet_service_type, dtype: int64

In [86]:
df.churn.value_counts()

No     5174
Yes    1869
Name: churn, dtype: int64

In [87]:
df.churn.value_counts('internet_service_type')

No     0.73463
Yes    0.26537
Name: churn, dtype: float64

In [89]:
df.gender.groupby

Male      0.504756
Female    0.495244
Name: gender, dtype: float64

In [94]:
df.groupby('phone_service').churn.agg('count')

phone_service
No      682
Yes    6361
Name: churn, dtype: int64

In [96]:
df.groupby('senior_citizen').churn.agg('count')

senior_citizen
0    5901
1    1142
Name: churn, dtype: int64

In [97]:
df.groupby('contract_type').churn.agg('count')

contract_type
Month-to-month    3875
One year          1473
Two year          1695
Name: churn, dtype: int64

In [ ]:
contract_type

In [81]:
df = df.drop_duplicates()
df

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,...,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,...,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,...,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2,3,1,9962-BFPDU,Female,0,Yes,Yes,1,Yes,...,No internet service,No internet service,No internet service,No,20.05,20.05,No,Month-to-month,None,Mailed check
7039,2,3,1,9967-ATRFS,Female,0,No,No,19,Yes,...,No internet service,No internet service,No internet service,No,19.90,367.55,No,Month-to-month,None,Mailed check
7040,4,3,1,9970-QBCDA,Female,0,No,No,6,Yes,...,No internet service,No internet service,No internet service,No,19.70,129.55,No,Month-to-month,None,Credit card (automatic)
7041,2,3,1,9975-SKRNR,Male,0,No,No,1,Yes,...,No internet service,No internet service,No internet service,No,18.90,18.9,No,Month-to-month,None,Mailed check


In [82]:
df['total_charges'].value_counts()
#we have 11 values that have an empty string:

df['total_charges'] = df['total_charges'].str.strip()
df['total_charges'] = df['total_charges'].replace('', 0)
df['total_charges'].value_counts()
df['total_charges'] = df['total_charges'].astype('float64')
df['total_charges']

0       5957.90
1       2460.55
2       2560.10
3        825.40
4       6316.20
         ...   
7038      20.05
7039     367.55
7040     129.55
7041      18.90
7042      85.50
Name: total_charges, Length: 7043, dtype: float64

## Sample questions

### Are customers with DSL more or less likely to churn?
### What month are customers most likely to churn and does that depend on their contract type?
### Is there a service that is associated with more churn than expected?
### Do customers who churn have a higher average monthly spend than those who don't?

In [ ]:
#preliminary clean delco:

df = df.drop_duplicates()  #Drop duplicates
#11 values are blank strings for total charges representing 11 customers at 0 tenure
df['total_charges'] = df['total_charges'].str.strip() #eliminate white space
df['total_charges'] = df['total_charges'].replace('', 0) #replace empty values as 0
df['total_charges'] = df['total_charges'].astype('float64')